# 1. Introduction to Convolution

In [20]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

In [21]:
#device = torch.device("mps")
device = torch.device("cpu")

When dealing with images, the **traditional neural networks** have the following problems:

1. too many pixels, **too many weights**
2. **shifted** images will be ahrd to recognize
3. **correlation** between pixles cannot be learnt

## Convolution

The **`convolution`** between two functions $f, g: \mathbb{R}^d \to \mathbb{R}$ is defined as:

$$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}$$

It measures the **overlap** between $f$ and $g$ when one function is **“flipped” and shifted** by $x$. 

When the functions are **discrete**, we have:

$$(f * g)(i) = \sum_a f(a) g(i-a)$$

For **2-dimensional tensors**, we have a corresponding sum with indices $(a,b)$ for $f$ and $(a-i,b-j)$ for $g$:

$$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i-a, j-b)$$

## Cross-Correlation

The **convolutional layer** is actually performing **`cross-correlation`** calculations instead of convolutions.

Normally, an image is represented by a **3-dimensional** tensor. For example, a $10×10×3$ tensor represents an image with **$10×10$ pixels** and **$3$ colors** (RGB).

Let's now ignore the **color channel** and just look at **cross-correlation** between the 2-dimensional tensor and the **kernel**:

![ ](http://d2l.ai/_images/correlation.svg)

The first output is obtained by carrying out the following calculation:

$$0×0+1×1+3×2+4×3=19$$

Similarly, the other outputs are obatained by **sliding** (from left to right, from top to bottom) the kernel:

$$
1\times0+2\times1+4\times2+5\times3=25\\
3\times0+4\times1+6\times2+7\times3=37\\
4\times0+5\times1+7\times2+8\times3=43
$$

We can implement this calculation with the following code:

```python 
Y[i,j] = (X[i:i+K_h,j:j+K_w]*K).sum()
```

In the above example, the ouput tensor is often referred as the **feature map** and the shaded area in the input tensor is called the **receptive field** of the output $19$.

As you can see, the original tensor is **shrinked**. For a $n_h \times n_w$ input and a $k_h \times k_w$ kernel, the size of the output is given as

$$(n_h-k_h+1) \times (n_w-k_w+1)$$

In [22]:
def corr2d(X, K):
    K_h, K_w = K.shape
    Y = torch.zeros((X.shape[0]-K_h+1, X.shape[1]-K_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+K_h,j:j+K_w]*K).sum()
    return Y

In [23]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## Convolutional Layers

A **`convolutional layer`** first **cross-corrlates** the input and the kernel and then **adds a scalar bias** to produce an output.

In [24]:
class Conv2D(nn.Module):
    
    def __init__(self, kernel_size):
        super().__init__()
        self.weights = nn.Parameter(torch.rand(kernel_size))     #initialize wthe eights (kernel)
        self.bias = nn.Parameter(torch.zeros(1))                 #initialize the scalar bias
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## Learning a Convolutional Kernel

Let's use backpropagation to learn the optimal kernel for a convolutional layer:

In [25]:
X = torch.ones((6, 8))
X[:, 2:6] = 0

K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)

In [26]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1,2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 14.254
epoch 4, loss 3.940
epoch 6, loss 1.296
epoch 8, loss 0.477
epoch 10, loss 0.187


In [27]:
conv2d.weight.data

tensor([[[[ 1.0293, -0.9416]]]])

To perform **convolution**, we need to **flip** the two-dimensional kernel tensor both **horizontally and vertically**, and then **cross-correlate** it with the input tensor.

However, since the **convolutional kernels** are learn from the input data, performing **convolution** or **cross-correlation** will not affect the output of the convolutional layer.

That is, if we use **convolution** instead of **cross-correlation** in learning, we will end up with a **kernel** that is equivalent to the kernel learned with cross-correlation but just flipped horizontally and veritcally.

## Padding

Convolution usually **shrinks** the input data from $n_h \times n_w$ to $(n_h-k_h+1) \times (n_w-k_w+1)$:

![ ](http://d2l.ai/_images/correlation.svg)

When the input data is convolved **multiple times**, we tend to **lose pixels on the perimeter** of the image. 

One straightforward solution to this problem is to **add extra pixels of filler around the boundary** of the input image, thus **increasing the effective size** of the image. Typically, we set the values of the extra pixels to **zero**.

This is known as **`padding`**:

![](http://d2l.ai/_images/conv-pad.svg)

If we add a total of **$p_h$ rows** of padding (roughly half on top and half on bottom) and a total of **$p_w$ columns** of padding (roughly half on the left and half on the right), the output shape becomes:

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)$$

Therefore, to **maintain the size** of the input, we need to set $p_h=k_h-1$ and $p_w=k_w-1$. 

We usually use **odd** number of **kernel size** for the following benefits:

1. equal padding on both sides of the input
2. the output $Y[i,j]$ is the cross-correlation between the input window centered on $X[i, j]$ and the kernel



In [29]:
def comp_conv2d(conv2d, X):
    batch_size, n_channel = 1, 1
    X = X.reshape((batch_size,n_channel)+X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

In [30]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [31]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5,3), padding=(2,1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## Stride

We refer to the number of rows and columns **traversed per slide** by the convolution kernel as **`stride`**.

We use **larger strides** for better **computational efficiency** or when we wish to **downsample**. That is, the convolution kernel moves **more than one element** at a time. This is particularly useful when the kernel is large.

For example, we set the horizontal stride $s_w=2$ and the vertical stride $s_h=3$, we have the following:

![](http://d2l.ai/_images/conv-stride.svg)

In general, when the stride for the height is $s_h$ and the stride for the width is $s_w$, the output shape is given as:

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor$$

In [32]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

## Multiple Input Channels

When we have **multiple input channels**, we will first cross-correlates the 2D input tensors with their corresponding 2D kernels (same channel), then add the results to obtain the output tensor:

![](http://d2l.ai/_images/conv-multi-in.svg)

In [33]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x,k) for x,k in zip(X,K))

In [34]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                 [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## Multiple Output Channels

To obtain **multiple output channels**, we can create a kernel with shape $c_i\times k_h\times k_w$ for each output channel, then concatenate them to obtain the convolution kernel with shape $c_o\times c_i\times k_h\times k_w$.

In [35]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [36]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [37]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1 \times 1$ Convolutional Layer

A **$1 \times 1$ convolutional layer** will not capture the cross-correlations between pixels, but instead it outputs the **weighted linear combinations** of the same pixels in **different channels**:

![](http://d2l.ai/_images/conv-1x1.svg)

Thus, the $1 \times 1$ convolutional layer requires **$c_o\times c_i$ weights** (plus the bias).

In [38]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [40]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

float(torch.abs(Y1 - Y2).sum()) < 1e-6

True

## Pooling

The **`pooling layers`** is used for two purposes:

1. to **mitigate the sensitivity** of convolutional layers to **location**
2. to **spatially downsampling** representations

Similar to convolutional layers, pooling operators consist of a **fixed-shape window** that slides over the **entire region** of the input with a specified **stride size**. 

However, the pooling layer contains **no parameters** but instead is deterministic, typically calculating either the **maximum** or the **average** value of the elements in the pooling window.

For example, a **maximum pooling** layer does the following:

![](http://d2l.ai/_images/pooling.svg)

where we have:

$$
\max(0, 1, 3, 4)=4\\
\max(1, 2, 4, 5)=5\\
\max(3, 4, 6, 7)=7\\
\max(4, 5, 7, 8)=8\\
$$

A $p_w \times p_h$ pooling layer **shrinks** the input tensor with shape $n_h \times n_w$ to $(n_h-p_h+1) \times (n_w-p_w+1)$.

In [41]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0]-p_h + 1, X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_w].mean()
    return Y

In [42]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [43]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

We can also change the **padding** and **stride** of the pooling layer:

In [48]:
pool2d = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)

In [49]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

When there are multiple input channels, the pooling layer outputs the same number of channels:

In [50]:
X = torch.cat((X,X+1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [51]:
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])